# The Basics: Models, locations & agents

After the installation of popy, the first thing we have to do is to import the package in order to use it:

In [1]:
import popy

## Model

In Popy, `Model` is the most important object class because almost everything happens inside - or at least in regard to - a `Model`.
You can imagine a `Model` as the world where all the entities we create live.
Therefore, everthing you can do in Popy starts with creating an instance of `popy.Model`.

In [2]:
model = popy.Model()

A `Model` has three very important attributes.

`Model.g` is a network-graph which stores all agents, locations and their relations to each other as a bipartite network:

In [3]:
model.g

`Model.agents` returns a list of all agents stored in `Model.g`:

In [4]:
model.agents

AgentList (0 objects)

`Model.locations` returns a list of all locations stored in `Model.g`:

In [5]:
model.locations

LocationList (0 objects)

## Agents and locations

Next, we create our first agent:

In [6]:
agent = popy.Agent(model)

Whenever a new instance of an `Agent` is created, it gets automatically added to the given instance of `Model`.
This means that the created `Agent` instance is represented as a node in `model.g` and gets returned in `model.agents`:

In [7]:
model.g.nodes

NodeView((1,))

In [8]:
model.agents

AgentList (1 object)

In [9]:
model.agents[0]

Agent (Obj 1)

The same applies to locations:

In [10]:
location = popy.Location(model)

In [11]:
model.g.nodes

NodeView((1, 2))

In [12]:
model.locations

LocationList (1 object)

In programming, often the best thing you can do to understand what is going on is to look at things.
In Popy, the `NetworkInspector` helps you to understand the network you are building.
For instance, using the `NetworkInsepctor`, you can visualize the bipartite network of agents and locations:

In [13]:
inspector = popy.NetworkInspector(model)
inspector.plot_bipartite_network()

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : line_alpha='weight' [no close matches] {renderer: GlyphRenderer(id='p1053', ...)}


To be honest, the above is not really a network, because there are only two nodes without any edge.
However, the graph gives an intuitive overview over the entities in your model.

## Connecting agents and locations

Our model has one agent and one location.
Let's create an edge between those two entities by *adding* the agent to the location:

In [14]:
location.add_agent(agent)

In [15]:
inspector.plot_bipartite_network()

The attribute `location.agents` 

In [16]:
location.agents

AgentList (1 object)

In [17]:
agent.locations

LocationList (1 object)

Let's add some more agents to the model using a For-Loop:

In [18]:
for _ in range(3):
    popy.Agent(model)

As you can see, we do not even have to assign the created instances of `Agent` to a variable because their are added to the model automatically during the initialization.
Now we have 4 agents in our model:

In [19]:
model.agents

AgentList (4 objects)

Let's add an additional location.
But now we create our own location class `School` which inherits from `popy.Location`.
You will understand the purpose of this later.

In [20]:
class School(popy.Location):
    pass

School(model)

School (Obj 6)

In [21]:
model.locations

LocationList (2 objects)

Let's have a look at the bipartite network graph:

In [22]:
inspector.plot_bipartite_network()

Because there is still only one edge, in the next step we assign each agent to one location and make sure that each location has not more than 2 agents:

In [23]:
for location in model.locations:
    for agent in model.agents:
        if len(location.agents) < 2 and len(agent.locations) == 0:
            location.add_agent(agent)

In [24]:
model.g.nodes[1]


{'bipartite': 0, '_obj': Agent (Obj 1)}

In [25]:
inspector.plot_bipartite_network()

In [26]:
agent = model.agents[0]
for location in model.locations:
    if agent not in location.agents:
        location.add_agent(agent)

In [27]:
inspector.plot_bipartite_network()

By the way: We can easily plot the netowrk on the agent-level:

In [28]:
inspector.plot_agent_network()

## Finding neighbors and shared locations

From the perspective of an agent, it is very important to know with whom the agent is connected in the network.
As popy works with bipartite networks, technically agents are only connected with locations.
As you might already know, agents can access all connected locations via `Agent.locations`:




In [29]:
agent = model.agents[0]
agent.locations

LocationList (2 objects)

As locations are mainly a helping tool to connect agents, we often want to know which other agents are connected to the same locations as a specific agent.
Every agent can access those other agents who share a location with this specific agent via `Agent.neighbors()`:

In [30]:
agent.neighbors()

AgentList (3 objects)

If we want to find only those *neighbors* who share a specific type of location with the agent, we can use the argument `location_classes` to filter by types of locations.
In the following we only want *neighbors* who share a location of the type `School` with the agent:

In [31]:
agent.neighbors(location_classes=[School])

AgentList (2 objects)

By using different types of locations we can organize the relations of an agent to other agents and then access *neighbors* with a certain relation to the agent as we just did it above.

Sometimes we need to access all the locations an agent shares with another agent.
This can be done using `Agent.shared_locations()`:

In [32]:
agent2 = model.agents[-1]

agent.shared_locations(agent2)

LocationList (1 object)

In [33]:
agent.shared_locations(agent2)[0]

School (Obj 6)

`Agent.shared_locations()` also has the argument `location_classes` to filter by the type of the location:

In [34]:
agent.shared_locations(agent2, location_classes=[School])

LocationList (1 object)

In [35]:
agent.shared_locations(agent2, location_classes=(School))

TypeError: 'type' object is not iterable